In [294]:
import pandas as pd
import unicodedata

# 0 Funcoes

In [295]:
# Como estamos importando de um excel, vamos ter que ajustar o nome das colunas
def normalizar_colunas(df):
    def remover_acentos(texto):
        return ''.join(
            c for c in unicodedata.normalize('NFKD', texto)
            if not unicodedata.combining(c)
        )

    df = df.copy()
    df.columns = [
        remover_acentos(col)
            .lower()
            .replace(' ', '_')
        for col in df.columns
    ]

    return df


In [296]:
# Como vamos juntar alguns dataframes, é melhor que estejam com os mesmos nomes algumas colunas
def renomear_colunas(df, mapa_colunas):
    """
    Parâmetros:
    df (pd.DataFrame): DataFrame original
    mapa_colunas (dict): {'nome_antigo': 'nome_novo'}

    Retorna:
    pd.DataFrame: DataFrame com colunas renomeadas
    """
    df = df.copy()

    # Aplica somente às colunas que existem no DataFrame
    mapa_valido = {
        col_antiga: col_nova
        for col_antiga, col_nova in mapa_colunas.items()
        if col_antiga in df.columns
    }

    df.rename(columns=mapa_valido, inplace=True)
    return df


In [297]:
"""
Como os dataframes tem colunas diferentes, na hora de juntar essa função irá ajudar.
Iremos adicionar colunas nos outros dataframes para que a junção possa ocorrer (as colunas terão dasdos vazios)
"""
def adaptar_dataframe(df, colunas_base, origem, lista_ids):
    df=df.copy()

     # filtra apenas os IDs desejados
    df = df[df['ra'].isin(lista_ids)]

    # adiciona colunas que faltam
    for col in colunas_base:
        if col not in df.columns:
            df[col] = pd.NA

    # mantém apenas as colunas do principal
    df = df[colunas_base]

    # cria coluna de origem
    df['ano_dataframe'] = origem

    return df


In [298]:
# Função correção de tipo de colunas
def corrigir_dados(tipo, dataframe, colunas):
    """
    Corrige o tipo de dados de colunas de um DataFrame.

    Parâmetros:
    tipo (str): tipo alvo ('int', 'float', 'str', 'data')
    dataframe (pd.DataFrame): DataFrame original
    colunas (list ou str): coluna ou lista de colunas

    Retorna:
    pd.DataFrame: DataFrame com colunas corrigidas
    """
    df = dataframe.copy()

    if isinstance(colunas, str):
        colunas = [colunas]

    match tipo.lower():
        case 'int':
            for col in colunas:
                df[col] = pd.to_numeric(df[col], errors='coerce').astype('Int64')

        case 'float':
            for col in colunas:
                df[col] = pd.to_numeric(df[col], errors='coerce')

        case 'str':
            for col in colunas:
                df[col] = df[col].astype(str).str.strip()

        case 'data' | 'datetime':
            for col in colunas:
                df[col] = pd.to_datetime(df[col], errors='coerce')

        case _:
            raise ValueError(
                f"Tipo '{tipo}' não suportado. "
                "Use: int, float, str, data"
            )

    return df

In [299]:
def colunas_totalmente_vazias(df):
    """
    Identifica colunas que possuem apenas valores vazios e deleta

    Parâmetros:
    df (pd.DataFrame): DataFrame de entrada
    remover (bool): se True, remove as colunas vazias

    Retorna:
    pd.DataFrame:
        - DataFrame sem colunas vazias (remover=True)
    """
    df_tmp = df.copy()

    # considera strings vazias como NaN
    df_tmp = df_tmp.replace(r'^\s*$', pd.NA, regex=True)

    colunas_vazias = [
        col for col in df_tmp.columns
        if df_tmp[col].isna().all()
    ]

    return df_tmp.drop(columns=colunas_vazias)


In [300]:
def arredondar_floats(df, casas=2):
    """
    Arredonda todas as colunas float de um DataFrame.

    Parâmetros:
    df (pd.DataFrame): DataFrame de entrada
    casas (int): número de casas decimais (padrão = 2)

    Retorna:
    pd.DataFrame: DataFrame com floats arredondados
    """
    df = df.copy()

    colunas_float = df.select_dtypes(include=['float', 'float64', 'float32']).columns

    df[colunas_float] = df[colunas_float].round(casas)

    return df

# 1 Importando os dados

In [301]:
arquivo = r"https://raw.githubusercontent.com/vbomura/tc5/0afbe2d22ee4658c81a7470b2360a2aef3cd032e/Base_Passos_Magicos/BASE%20DE%20DADOS%20PEDE%202024%20-%20DATATHON.xlsx"

# Pegando o dado de cada aba
base_2022 = pd.read_excel(arquivo, sheet_name="PEDE2022")
base_2023 = pd.read_excel(arquivo, sheet_name="PEDE2023")
base_2024 = pd.read_excel(arquivo, sheet_name="PEDE2024")

## Ajustes nas colunas

In [302]:
# Vamos padronizar os nomes das colunas
base_2022 = normalizar_colunas(base_2022)
base_2023 = normalizar_colunas(base_2023)
base_2024 = normalizar_colunas(base_2024)

In [303]:
# Renomeando algumas colunas das bases
mapa = {
    'pedra_23': 'ignorar' #tem uma coluna "repetida no dataframe, vamos ignorar ela pq não tem a info que precisamos"
    ,'pedra_2023': 'pedra_23'
    ,'inde_23': 'ignorar2'
    ,'inde_2023': 'inde_23'
}
base_2023 = renomear_colunas(base_2023, mapa)

mapa = {
    'portug': 'por'
    ,'ingles': 'ing'
    ,'matem': 'mat'
    ,'defas': 'defasagem'
    ,'idade_22': 'idade'
    
}
base_2022 = renomear_colunas(base_2022, mapa)


## Juntando os dataframes

In [304]:
# Vamos juntar os dataframes, mas primeiro vamos evitar juntar casos que já estão 2024
ra_principal = set(base_2024['ra'])

base_2023_filtrado = base_2023[
    (~base_2023['ra'].isin(ra_principal)) #alguns valores estão com problemas,
]

ra_principal.update(base_2023_filtrado['ra'].unique()) #marca os de 2023 para evitar duplicados que tem apenas entre 23 e 22

base_2022_filtrado = base_2022[
    (~base_2022['ra'].isin(ra_principal))
]

# Vamos guardar os id em listas para usar depois
ra_2023_diferentes = base_2023_filtrado['ra'].unique().tolist()
ra_2022_diferentes = base_2022_filtrado['ra'].unique().tolist()

In [305]:
# listamos as colunas do dataframe principal (ano de 2024)
colunas_finais = list(base_2024.columns) + ['ano_dataframe']

# Colocamos as colunas nos dataframes, para que eles fiquem 'parecidos' com 2024, e também uma coluna para saber de qual ano veio a info
base_2024['ano_dataframe'] = 2024
base_2023 = adaptar_dataframe(base_2023,colunas_finais, 2023, ra_2023_diferentes)
base_2022 = adaptar_dataframe(base_2023,colunas_finais, 2022, ra_2022_diferentes)

In [306]:
# Após todos os tratamentos, juntamos tudo
df_unificado = pd.concat(
    [base_2024, base_2023, base_2022],
    ignore_index=True
)

## Limpeza e correção dos dados dataframe

In [307]:
# Primeiro vamos corrigir o tipo de algumas colunas
colunas_correcao = ['inde_2024', 'inde_22', 'inde_23']
df_unificado = corrigir_dados('float', df_unificado, colunas_correcao) #corrigi os dados que deveriam ser float

colunas_correcao = ['data_de_nasc']
df_unificado = corrigir_dados('data', df_unificado, colunas_correcao) #corrigi os dados que deveriam ser data

colunas_correcao = ['idade']
df_unificado = corrigir_dados('int', df_unificado, colunas_correcao) #corrigi os dados que deveriam ser data

In [308]:
# Retirando colunas com valores apenas vazios, após análises não vi valor em manter ou tentar preencher
df_unificado = colunas_totalmente_vazias(df_unificado)
df_unificado = df_unificado.drop(columns=['avaliador1','avaliador2','avaliador3','avaliador4','avaliador5','avaliador6'], errors='ignore') #essas colunas não trazem nenhum valor para a análise

In [309]:
# Correções e ajustes no valores da tabela
df_unificado = arredondar_floats(df_unificado)

# Testes

In [310]:
df_unificado.sample(50)

,ra,fase,inde_2024,pedra_2024,turma,nome_anonimizado,data_de_nasc,idade,genero,ano_ingresso,...,por,ing,ipv,ian,fase_ideal,defasagem,escola,ativo/_inativo,ativo/_inativo.1,ano_dataframe
141,RA-1353,ALFA,6.45,Agata,ALFA Q - G2/G3,Aluno-1353,2014-10-19,10,Feminino,2024,...,2.5,NaN,6.08,5.0,Fase 1 (3° e 4° ano),-1,EE Luiz Schunck,Cursando,Cursando,2024
1321,RA-410,FASE 3,NaN,NaN,3K,Aluno-410,2008-06-12,<NA>,Masculino,2018,...,4.8,4.8,6.04,5.0,Fase 4 (9° ano),-1,NaN,NaN,NaN,2023
273,RA-672,1H,4.21,Quartzo,1H,Aluno-672,2013-08-26,11,Feminino,2021,...,0.0,NaN,4.50,5.0,Fase 2 (5° e 6° ano),-1,EM Etelvina Delfim Simões,Cursando,Cursando,2024
26,RA-885,ALFA,8.21,Topázio,ALFA D - G2/G3,Aluno-885,2015-10-30,9,Masculino,2023,...,7.0,NaN,8.28,5.0,Fase 1 (3° e 4° ano),-1,EE Dom Pedro Villas Boas de Souza,Cursando,Cursando,2024
949,RA-1612,5F,7.01,Ametista,5F,Aluno-1612,2006-08-11,18,Masculino,2024,...,6.5,7.0,6.94,5.0,Fase 7 (3° EM),-2,EE Joaquim Mendes Feliz,Cursando,Cursando,2024
807,RA-1570,4B,7.89,Ametista,4B,Aluno-1570,2008-08-06,16,Feminino,2024,...,9.0,9.5,8.67,5.0,Fase 5 (1° EM),-1,EE Alexandre Rodrigues Nogueira,Cursando,Cursando,2024
517,RA-777,2N,7.56,Ametista,2N,Aluno-777,2014-05-07,10,Feminino,2022,...,5.5,NaN,7.72,10.0,Fase 2 (5° e 6° ano),0,EE Helio Luiz Dobrochinski Prof,Cursando,Cursando,2024
714,RA-1542,3M,6.09,Agata,3M,Aluno-1542,2008-09-18,16,Feminino,2024,...,3.5,5.0,6.97,5.0,Fase 5 (1° EM),-2,EE Donizetti Aparecido Leite Professor,Cursando,Cursando,2024
250,RA-874,1E,7.02,Ametista,1E,Aluno-874,2013-11-17,10,Masculino,2023,...,9.5,NaN,7.92,5.0,Fase 2 (5° e 6° ano),-1,EE Chácara Florida II,Cursando,Cursando,2024
972,RA-1617,5M,4.93,Quartzo,5M,Aluno-1617,2008-06-08,16,Feminino,2024,...,6.0,NaN,6.12,5.0,Fase 6 (2° EM),-1,EE Donizetti Aparecido Leite Professor,Cursando,Cursando,2024


In [311]:
df_unificado.dtypes

# df_unificado['destaque_ieg'].describe()
# df_unificado['destaque_ieg'].sample(50)
# df_unificado['cg'].isna().sum()
# df.nunique().sort_values()

ra                               object
fase                             object
inde_2024                       float64
pedra_2024                       object
turma                            object
nome_anonimizado                 object
data_de_nasc             datetime64[ns]
idade                             Int64
genero                           object
ano_ingresso                      int64
instituicao_de_ensino            object
pedra_20                         object
pedra_21                         object
pedra_22                         object
pedra_23                         object
inde_22                         float64
inde_23                         float64
no_av                           float64
iaa                             float64
ieg                             float64
ips                             float64
ipp                             float64
ida                             float64
mat                             float64
por                             float64
